In [2]:
# Install library (jika belum terinstall di environment)
# !pip install insightface onnxruntime-gpu scikit-image facenet_pytorch
# %pip install numpy seaborn facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 8.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 107.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 111.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from facenet_pytorch import InceptionResnetV1
from sklearn.metrics import classification_report, confusion_matrix
from pathlib import Path

# Setup Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device yang digunakan: {device}")   

Device yang digunakan: cpu


## Wajib Restart kernel

## Load Dataset Test

In [2]:
# --- KONFIGURASI ---
MODEL_PATH = "best_inception_resnet.pth" # Pastikan path ini sesuai lokasi file .pth Anda
TEST_DIR = "Test" # Arahkan ke folder test set Anda. (Bisa gunakan folder validasi jika tidak ada folder test khusus)
IMG_SIZE = 224
BATCH_SIZE = 32

In [3]:
# Transformasi (Hanya Resize & Normalize, tanpa Augmentasi)
test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

print(f"Model Path: {MODEL_PATH}")
print(f"Test Directory: {TEST_DIR}")

Model Path: best_inception_resnet.pth
Test Directory: Test


In [4]:
# 1. Load Checkpoint
if torch.cuda.is_available():
    checkpoint = torch.load(MODEL_PATH)
else:
    checkpoint = torch.load(MODEL_PATH, map_location=torch.device('cpu'))

# 2. Ambil informasi kelas dari checkpoint
class_names = checkpoint['class_names']
num_classes = len(class_names)
print(f"Jumlah Kelas: {num_classes}")
print(f"Nama Kelas: {class_names}")

# 3. Inisialisasi Arsitektur Model (InceptionResnetV1)
model = InceptionResnetV1(
    pretrained=None, # Kita tidak perlu download weight pretrain lagi karena akan di-load dari file lokal
    classify=True,
    num_classes=num_classes,
    dropout_prob=0.5
)

# 4. Load State Dict (Bobot Model) ke Arsitektur
model.load_state_dict(checkpoint['state_dict'])
model.to(device)
model.eval() # Set ke mode evaluasi (matikan dropout/batchnorm update)

print("Model berhasil dimuat dan siap untuk evaluasi.")

FileNotFoundError: [Errno 2] No such file or directory: 'best_inception_resnet.pth'

In [ ]:
# Load Test Dataset
try:
    test_dataset = datasets.ImageFolder(TEST_DIR, transform=test_transform)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    print(f"Ditemukan {len(test_dataset)} gambar untuk evaluasi.")
except FileNotFoundError:
    print(f"Error: Folder {TEST_DIR} tidak ditemukan. Mohon cek path dataset.")

In [ ]:
y_true = []
y_pred = []

print("Mulai Evaluasi...")

with torch.no_grad(): # Matikan gradien untuk menghemat memori
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        
        # Ambil kelas dengan probabilitas tertinggi
        _, preds = torch.max(outputs, 1)

        # Simpan hasil ke list (pindahkan ke CPU dulu)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("Evaluasi Selesai.")

In [ ]:
# Menghitung Precision, Recall, F1-Score
print("--- Classification Report ---")
report = classification_report(
    y_true, 
    y_pred, 
    target_names=class_names, 
    zero_division=0
)
print(report)

In [ ]:
# Membuat Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

# Plotting
plt.figure(figsize=(12, 10))
sns.heatmap(
    cm, 
    annot=True, # Tampilkan angka
    fmt='d',    # Format angka integer
    cmap='Blues', 
    xticklabels=class_names, 
    yticklabels=class_names
)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix - Test Set')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()